## Just for 2007 - 2017 CDBG for now

In [1]:
import pandas as pd

In [2]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

In [3]:
path = '/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/'
df_house = pd.read_csv(path + 'ToDatabase/TotalHouse_v04.csv')

In [4]:
df_cdbg = pd.read_csv(path + 'Raw-Data/test_cdbg_2007-2017.csv')

In [5]:
df_house.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)
df_cdbg.drop('Unnamed: 0', inplace=True, axis = 1)

In [6]:
df_cdbg.head()

,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,SANDS DRIVE,,0.00,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
1,421 W SOCIETY AVE,,124559.66,522,11,08/22/2007,2006,DEMOLITION,Completed,4/6/2010
2,525 5TH AVE,,15016.43,523,8,08/22/2007,2006,PUBLIC FACILITIES,Completed,8/16/2011
3,PO BOX 448,DAYCARE SERVICES FOR SENIORS DIAGNOSED WITH AL...,14950.95,539,15,10/04/2007,2007,PUBLIC SERVICE,Completed,8/6/2010
4,400 EBONY LN,Drainage and street improvements made to the 4...,46642.04,546,11,10/04/2007,2007,PUBLIC FACILITIES,Completed,11/29/2010


In [7]:
df_house.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
0,21204,0000Q/00025/009,1707 EDGERLY AVE,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN
1,26763,0000U/00006/002,703 DARTMOUTH LN,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN
2,21680,0000H/00047/005,1115 W WADDELL AVE,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0,NaN,NaN
3,28935,0000X/00003/034,313 FORRESTAL LN,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0,NaN,NaN
4,22320,00240/00003/024,1125 JOEL DR,2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0,NaN,NaN


In [8]:
s_cdbg_id = df_cdbg['IDISActivityID'] #pull ids
s_house_id = df_house['IDIS Activity ID'] #pull ids

In [9]:
s_cdbg_id.shape #shapes

(1111,)

In [10]:
s_house_id.shape #shapes

(850,)

#### We want to merge the two databases based upon id... we need to do some preliminary exploration first

In [11]:
s_cdbg_id.value_counts().head() # so multiple projects have the same id which is not what we anticipated

466     5
771     5
523     5
1146    5
547     4
Name: IDISActivityID, dtype: int64

In [12]:
s_cdbg_id.value_counts().tail()

1042    1
1043    1
1057    1
1096    1
445     1
Name: IDISActivityID, dtype: int64

In [13]:
df_cdbg[s_cdbg_id == 466] # but luckily these are just duplicate records

,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,SANDS DRIVE,,0.0,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
137,SANDS DRIVE,,0.0,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
405,SANDS DRIVE,,0.0,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
514,SANDS DRIVE,,0.0,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
644,SANDS DRIVE,,0.0,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012


#### Let's see what happens when we drop duplicate records

In [14]:
df_cdbg = df_cdbg.drop_duplicates()

In [15]:
s_cdbg_id = df_cdbg['IDISActivityID']

In [16]:
s_cdbg_id.value_counts().head() # so now each id has only one record attached to it

1023    1
1401    1
1426    1
1425    1
1424    1
Name: IDISActivityID, dtype: int64

#### Let's check the intersection of ids found in the pr03s drawdown and the ids found in the housing projects list

In [17]:
cdbg_notin_house = df_cdbg[~s_cdbg_id.isin(s_house_id)]

In [18]:
house_notin_cdbg = df_house[~s_house_id.isin(s_cdbg_id)] # these are the more worrisome ones

#### how many of these are home grants? Let's try this again, but after filtering for only cdbg in our dataset

In [19]:
df_house_cdbg = df_house[df_house['Program'] == "CDBG"]

In [20]:
df_house_cdbg.index = range(len(df_house_cdbg))

In [21]:
s_house_id = df_house_cdbg['IDIS Activity ID']

In [22]:
cdbg_notin_house = df_cdbg[~s_cdbg_id.isin(s_house_id)]

In [23]:
house_notin_cdbg = df_house_cdbg[~s_house_id.isin(s_cdbg_id)]

In [24]:
cdbg_notin_house.shape

(173, 10)

In [25]:
house_notin_cdbg.shape # great! only one record... what is it?

(1, 14)

In [26]:
house_notin_cdbg

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
563,20726,NaN,514 PINE AVE,2017,22,Commercial Rehabilitation,1547,Open,CDBG,0.0,0.0,0.0,NaN,NaN


#### interesting that the project is open... let's ignore this one for now, and just match the ones that we can

In [27]:
df_cdbg.shape #indexes are same

(686, 10)

In [28]:
s_cdbg_id.shape

(686,)

In [29]:
df_cdbg.head()

,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,SANDS DRIVE,,0.00,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
1,421 W SOCIETY AVE,,124559.66,522,11,08/22/2007,2006,DEMOLITION,Completed,4/6/2010
2,525 5TH AVE,,15016.43,523,8,08/22/2007,2006,PUBLIC FACILITIES,Completed,8/16/2011
3,PO BOX 448,DAYCARE SERVICES FOR SENIORS DIAGNOSED WITH AL...,14950.95,539,15,10/04/2007,2007,PUBLIC SERVICE,Completed,8/6/2010
4,400 EBONY LN,Drainage and street improvements made to the 4...,46642.04,546,11,10/04/2007,2007,PUBLIC FACILITIES,Completed,11/29/2010


In [30]:
df_house_cdbg.iloc[0]['IDIS Activity ID']

578

In [31]:
(578 in s_cdbg_id.values)

True

In [32]:
df_cdbg[s_cdbg_id == 578]['Address'].index[0]

473

In [33]:
def match_ids(df_main, df_info, s_info): #potentially reusable but not really. main is full house, info is cdbg
    s_house_addr = []
    s_cdbg_addr = []
    s_initial_date = []
    s_status_date = []
    s_house_ind = [] # should be close to a range with one misstep
    s_cdbg_ind = []
    for i in range(len(df_main)):
        row = df_main.iloc[i]
        proj_id = row['IDIS Activity ID']
        if proj_id in s_info.values:
            record = df_info[s_info == proj_id] # brings the specific record where there is a match (should be unique)
            # might not be unique actually
            ind = record.index[0]
            s_cdbg_ind.append(ind)
            record = record.iloc[0]
            s_house_addr.append(row['Address'])
            s_cdbg_addr.append(record['Address'])
            s_initial_date.append(record['InitialFundingDate'])
            s_status_date.append(record['StatusDate'])
            s_house_ind.append(i)
    return pd.DataFrame({'HOUSE_ADDR' : s_house_addr, 'CDBG_ADDR' : s_cdbg_addr, 
                        'INIT_DATE' : s_initial_date, 'STAT_DATE' : s_status_date,
                        'HOUSE_INDEX' : s_house_ind, 'CDBG_INDEX' : s_cdbg_ind})

In [34]:
df_toadd = match_ids(df_house_cdbg, df_cdbg, s_cdbg_id)

In [35]:
df_toadd.shape

(563, 6)

In [36]:
df_toadd.head(5)

,HOUSE_ADDR,CDBG_ADDR,INIT_DATE,STAT_DATE,HOUSE_INDEX,CDBG_INDEX
0,1008 E RESIDENCE AVE,1008 E RESIDENCE AVE,03/08/2008,4/17/2008,0,473
1,1100 SWIFT ST,1100 SWIFT ST,06/03/2008,6/30/2008,1,504
2,1117 W WADDELL AVE,1117 W WADDELL AVE,06/02/2008,6/30/2008,2,497
3,1223 E BROAD AVE,1223 E BROAD AVE,03/08/2008,4/16/2008,3,470
4,1316 E 4TH AVE,1316 E 4TH AVE,06/03/2008,4/9/2009,4,160


In [37]:
mismatched = df_toadd[df_toadd['HOUSE_ADDR'] != df_toadd['CDBG_ADDR']]

In [38]:
mismatched.head()

,HOUSE_ADDR,CDBG_ADDR,INIT_DATE,STAT_DATE,HOUSE_INDEX,CDBG_INDEX
5,1419 W HIGHLAND AVE,1419 HIGHLAND AVE,05/30/2008,6/30/2008,5,493
39,705 16TH AVE,706 16TH AVE,06/05/2008,6/30/2008,39,164
41,UNKNOWN ADDRESS,SCATTERED SITES,10/09/2007,9/29/2009,41,10
43,230 S JACKSON ST,ALBANY COMMUNITY TOGETHER! (ACT!),06/10/2008,9/29/2009,43,19
44,314 N JEFFERSON ST,PO BOX 448,10/04/2007,8/6/2010,44,3


In [39]:
df_house_cdbg.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
0,26627,NaN,1008 E RESIDENCE AVE,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0,NaN,NaN
1,25739,NaN,1100 SWIFT ST,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.0,NaN,NaN
2,21679,NaN,1117 W WADDELL AVE,2007,5,Energy Efficiency,607,Completed,CDBG,3977.52,3977.52,0.0,NaN,NaN
3,5461,NaN,1223 E BROAD AVE,2007,7,Emergency Repairs,575,Completed,CDBG,4776.14,4776.14,0.0,NaN,NaN
4,25773,NaN,1316 E 4TH AVE,2007,7,Emergency Repairs,613,Completed,CDBG,6934.40,6934.40,0.0,NaN,NaN


In [40]:
df_cdbg.head()

,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,SANDS DRIVE,,0.00,466,6,09/28/2006,2005,HOUSING,Canceled,1/10/2012
1,421 W SOCIETY AVE,,124559.66,522,11,08/22/2007,2006,DEMOLITION,Completed,4/6/2010
2,525 5TH AVE,,15016.43,523,8,08/22/2007,2006,PUBLIC FACILITIES,Completed,8/16/2011
3,PO BOX 448,DAYCARE SERVICES FOR SENIORS DIAGNOSED WITH AL...,14950.95,539,15,10/04/2007,2007,PUBLIC SERVICE,Completed,8/6/2010
4,400 EBONY LN,Drainage and street improvements made to the 4...,46642.04,546,11,10/04/2007,2007,PUBLIC FACILITIES,Completed,11/29/2010


### Things to investigate

In [41]:
col1 = list(df_house_cdbg)

In [42]:
col2 = list(df_cdbg)

In [43]:
row1 = df_house_cdbg.iloc[5]

In [44]:
row2 = df_cdbg.loc[493]

In [45]:
pd.concat([row1, row2], axis = 0, ignore_index=False)

PrimaryID                                                         18199
Parcel ID                                                           NaN
Address                                             1419 W HIGHLAND AVE
Plan Year                                                          2007
IDIS Project                                                          5
Project                                               Energy Efficiency
IDIS Activity ID                                                    603
Activity Status                                               Completed
Program                                                            CDBG
Funded Amount                                                   3985.52
Draw Amount                                                     3985.52
Balance                                                               0
Multihome_Type                                                      NaN
Multihome                                                       

In [46]:
conflict_funding = pd.DataFrame(columns=col1+col2)
for house_ind, cdbg_ind in zip(mismatched.index, mismatched['CDBG_INDEX']):
    house_row = df_house_cdbg.iloc[house_ind]
    cdbg_row = df_cdbg.loc[cdbg_ind]
    if house_row['Funded Amount'] != cdbg_row['FundedAmount']:
        full = pd.concat([house_row, cdbg_row], axis=0)
        conflict_funding = conflict_funding.append(full, ignore_index=True)
conflict_funding.drop(['Parcel ID', 'IDIS Project', 'Program', 'PrimaryID'], axis=1, inplace=True)

In [47]:
conflict_funding

,Address,Plan Year,Project,IDIS Activity ID,Activity Status,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,1300 NEWTON RD,2014,Disposition,1228,Completed,12509.81,12509.81,0.0,NaN,NaN,230 S JACKSON ST STE 315 SUITE 315,,37529.44,1228,4,09/23/2014,2014,DISPOSITION,Completed,6/30/2015
1,1306 NEWTON RD,2014,Disposition,1228,Completed,12509.81,12509.81,0.0,NaN,NaN,230 S JACKSON ST STE 315 SUITE 315,,37529.44,1228,4,09/23/2014,2014,DISPOSITION,Completed,6/30/2015
2,2423 HARVEY RD,2014,Disposition,1228,Completed,12509.82,12509.82,0.0,NaN,NaN,230 S JACKSON ST STE 315 SUITE 315,,37529.44,1228,4,09/23/2014,2014,DISPOSITION,Completed,6/30/2015
3,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,42,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
4,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,43,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
5,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,44,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
6,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,45,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
7,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,46,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
8,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,47,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
9,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,48,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015


In [48]:
counts = conflict_funding.groupby('IDIS Activity ID').count()

In [49]:
sums = conflict_funding.groupby('IDIS Activity ID').sum() #hypothesis: these are same records but split up on one end
# Left is project dataset, right is drawdown reports

In [50]:
sums['Funded Amount'] * counts['Funded Amount']

IDIS Activity ID
1228    112588.32
1269         0.00
1286     67305.08
1336     56078.28
1394         0.00
1447     29625.00
Name: Funded Amount, dtype: float64

In [51]:
sums['FundedAmount']

IDIS Activity ID
1228    112588.32
1269    551824.00
1286     67305.08
1336     56078.28
1394    949432.00
1447     29625.00
Name: FundedAmount, dtype: float64

## hypothesis was almost correct--need to evaluate idis act 1269 and 1394

In [52]:
conflict_funding

,Address,Plan Year,Project,IDIS Activity ID,Activity Status,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,Address,Description,FundedAmount,IDISActivityID,IDISProjectId,InitialFundingDate,PlanYear,Project,Status,StatusDate
0,1300 NEWTON RD,2014,Disposition,1228,Completed,12509.81,12509.81,0.0,NaN,NaN,230 S JACKSON ST STE 315 SUITE 315,,37529.44,1228,4,09/23/2014,2014,DISPOSITION,Completed,6/30/2015
1,1306 NEWTON RD,2014,Disposition,1228,Completed,12509.81,12509.81,0.0,NaN,NaN,230 S JACKSON ST STE 315 SUITE 315,,37529.44,1228,4,09/23/2014,2014,DISPOSITION,Completed,6/30/2015
2,2423 HARVEY RD,2014,Disposition,1228,Completed,12509.82,12509.82,0.0,NaN,NaN,230 S JACKSON ST STE 315 SUITE 315,,37529.44,1228,4,09/23/2014,2014,DISPOSITION,Completed,6/30/2015
3,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,42,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
4,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,43,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
5,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,44,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
6,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,45,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
7,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,46,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
8,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,47,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015
9,2206 HABERSHAM RD,2014,Rental Rehabilitation,1269,Completed,NaN,NaN,NaN,APT,48,2206 HABERSHAM RD UNITS 41-60,Roof replacement to Windsor Arms Complex Units...,27591.20,1269,31,06/29/2015,2014,REHABILITATION,Completed,6/30/2015


In [53]:
df_house_cdbg[df_house_cdbg['Address'].str.contains("HABERSHAM")]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
22,30120,NaN,2500 HABERSHAM RD,2007,7,Emergency Repairs,576,Completed,CDBG,4746.14,4746.14,0.0,NaN,NaN
23,30109,NaN,2502 HABERSHAM RD,2007,5,Energy Efficiency,606,Completed,CDBG,2027.52,2027.52,0.0,NaN,NaN
185,25701,NaN,2206 HABERSHAM RD,2010,13,Demolition,940,Completed,CDBG,25565.92,25565.92,0.0,APT,31-40
388,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.20,27591.20,0.0,APT,41
389,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,42
390,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,43
391,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,44
392,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,45
393,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,46
394,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,47


In [54]:
551824.00 / 20 # this value matches the single record. So give everyone with idis 1269 same values

27591.2

In [55]:
df_house_cdbg[df_house_cdbg['IDIS Activity ID'] == 1269]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
388,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,41
389,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,42
390,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,43
391,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,44
392,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,45
393,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,46
394,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,47
395,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,48
396,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,49
397,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,NaN,NaN,NaN,APT,50


#### There are 21 values here--1 did not show up because the 20k value matched.

In [56]:
df_house_cdbg.loc[3,'IDIS Activity ID']

575

In [57]:
for i in range(len(df_house_cdbg)):
    record = df_house_cdbg.iloc[i]
    if record['IDIS Activity ID'] == 1269:
        df_house_cdbg.loc[i,'Funded Amount'] = 27591.2
        df_house_cdbg.loc[i,'Draw Amount'] = 27591.2
        df_house_cdbg.loc[i,'Balance'] = 0.0

/anaconda3/envs/cds/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [58]:
df_house_cdbg[df_house_cdbg['IDIS Activity ID'] == 1269]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
388,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,41
389,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,42
390,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,43
391,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,44
392,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,45
393,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,46
394,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,47
395,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,48
396,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,49
397,25701,NaN,2206 HABERSHAM RD,2014,31,Rental Rehabilitation,1269,Completed,CDBG,27591.2,27591.2,0.0,APT,50


### doing the same for 1394

In [59]:
sums

,Funded Amount,Draw Amount,Balance,FundedAmount
IDIS Activity ID,,,,
1228,37529.44,37529.44,0.0,112588.32
1269,0.00,0.00,0.0,551824.00
1286,33652.54,33652.54,0.0,67305.08
1336,9346.38,9346.38,0.0,56078.28
1394,0.00,0.00,0.0,949432.00
1447,9875.00,9875.00,0.0,29625.00


In [60]:
949432 / 20 # this matches

47471.6

In [61]:
df_house_cdbg[df_house_cdbg['IDIS Activity ID'] == 1394]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
409,5005,NaN,241 E OGLETHORPE BLVD,2014,34,Commercial Development Program,1394,Open,CDBG,47471.6,33325.08,14146.52,NaN,NaN
410,30589,NaN,242 E OGLETHORPE BLVD,2014,34,Commercial Development Program,1394,Open,CDBG,NaN,NaN,NaN,NaN,NaN
411,26513,NaN,243 E OGLETHORPE BLVD,2014,34,Commercial Development Program,1394,Open,CDBG,NaN,NaN,NaN,NaN,NaN
412,30589,NaN,244 E OGLETHORPE BLVD,2014,34,Commercial Development Program,1394,Open,CDBG,NaN,NaN,NaN,NaN,NaN
413,26509,NaN,245 E OGLETHORPE BLVD,2014,34,Commercial Development Program,1394,Open,CDBG,NaN,NaN,NaN,NaN,NaN
414,30589,NaN,246 E OGLETHORPE BLVD,2014,34,Commercial Development Program,1394,Open,CDBG,NaN,NaN,NaN,NaN,NaN
415,26514,NaN,247 E OGLETHORPE BLVD,2014,34,Commercial Development Program,1394,Open,CDBG,NaN,NaN,NaN,NaN,NaN
416,30589,NaN,248 E OGLETHORPE BLVD,2014,34,Commercial Development Program,1394,Open,CDBG,NaN,NaN,NaN,NaN,NaN
417,26515,NaN,249 E OGLETHORPE BLVD,2014,34,Commercial Development Program,1394,Open,CDBG,NaN,NaN,NaN,NaN,NaN
418,30589,NaN,250 E OGLETHORPE BLVD,2014,34,Commercial Development Program,1394,Open,CDBG,NaN,NaN,NaN,NaN,NaN


# Note for billy: check these in drawdown to make sure okay for draw and balance

In [62]:
for i in range(len(df_house_cdbg)):
    record = df_house_cdbg.iloc[i]
    if record['IDIS Activity ID'] == 1394:
        df_house_cdbg.loc[i,'Funded Amount'] = 47471.6
        df_house_cdbg.loc[i,'Draw Amount'] = 33325.08
        df_house_cdbg.loc[i,'Balance'] = 14146.52

## great--finally time to merge using toadd

In [63]:
df_toadd.head() # we can trust these comparisons

,HOUSE_ADDR,CDBG_ADDR,INIT_DATE,STAT_DATE,HOUSE_INDEX,CDBG_INDEX
0,1008 E RESIDENCE AVE,1008 E RESIDENCE AVE,03/08/2008,4/17/2008,0,473
1,1100 SWIFT ST,1100 SWIFT ST,06/03/2008,6/30/2008,1,504
2,1117 W WADDELL AVE,1117 W WADDELL AVE,06/02/2008,6/30/2008,2,497
3,1223 E BROAD AVE,1223 E BROAD AVE,03/08/2008,4/16/2008,3,470
4,1316 E 4TH AVE,1316 E 4TH AVE,06/03/2008,4/9/2009,4,160


In [64]:
df_toadd['CDBG_INDEX'].value_counts()

951     21
858     21
913      6
830      3
959      3
889      2
1019     1
264      1
266      1
265      1
262      1
263      1
261      1
260      1
259      1
258      1
257      1
267      1
269      1
268      1
255      1
270      1
271      1
273      1
274      1
275      1
276      1
277      1
278      1
279      1
280      1
281      1
282      1
283      1
256      1
251      1
254      1
230      1
194      1
195      1
197      1
202      1
205      1
207      1
208      1
1081     1
1090     1
226      1
227      1
228      1
229      1
233      1
253      1
234      1
235      1
237      1
240      1
242      1
243      1
244      1
245      1
246      1
247      1
248      1
249      1
286      1
285      1
288      1
181      1
289      1
333      1
365      1
366      1
367      1
368      1
369      1
370      1
371      1
374      1
376      1
377      1
1080     1
384      1
385      1
386      1
387      1
388      1
389      1
390      1
391      1
392      1

#### Actually, let's try running matchids again, after reindexing df_cdbg and s_cdbg_id

In [65]:
df_cdbg.index = range(len(df_cdbg))
s_cdbg_id = df_cdbg['IDISActivityID']

In [66]:
df_toadd = match_ids(df_house_cdbg, df_cdbg, s_cdbg_id)

In [67]:
df_toadd

,HOUSE_ADDR,CDBG_ADDR,INIT_DATE,STAT_DATE,HOUSE_INDEX,CDBG_INDEX
0,1008 E RESIDENCE AVE,1008 E RESIDENCE AVE,03/08/2008,4/17/2008,0,362
1,1100 SWIFT ST,1100 SWIFT ST,06/03/2008,6/30/2008,1,385
2,1117 W WADDELL AVE,1117 W WADDELL AVE,06/02/2008,6/30/2008,2,382
3,1223 E BROAD AVE,1223 E BROAD AVE,03/08/2008,4/16/2008,3,359
4,1316 E 4TH AVE,1316 E 4TH AVE,06/03/2008,4/9/2009,4,144
5,1419 W HIGHLAND AVE,1419 HIGHLAND AVE,05/30/2008,6/30/2008,5,378
6,1607 AVALON AVE,1607 AVALON AVE,03/08/2008,4/17/2008,6,370
7,1662 SUNNY LN,1662 SUNNY LN,06/03/2008,6/30/2008,7,384
8,1706 S JEFFERSON ST,1706 S JEFFERSON ST,06/06/2008,6/30/2008,8,150
9,1712 W HIGHLAND AVE,1712 W HIGHLAND AVE,05/30/2008,6/30/2008,9,380


In [68]:
df_toadd['CDBG_INDEX'].value_counts() # there are a couple that get assigned multiple times from drawdown which makes sense

526    21
597    21
561     6
498     3
605     3
537     2
209     1
214     1
213     1
212     1
211     1
685     1
208     1
207     1
215     1
205     1
204     1
203     1
206     1
218     1
216     1
217     1
201     1
219     1
220     1
221     1
223     1
224     1
226     1
227     1
228     1
229     1
230     1
231     1
202     1
199     1
200     1
161     1
165     1
166     1
167     1
168     1
169     1
172     1
173     1
174     1
176     1
178     1
180     1
181     1
182     1
183     1
184     1
185     1
186     1
187     1
189     1
191     1
192     1
193     1
194     1
195     1
196     1
197     1
198     1
232     1
235     1
236     1
279     1
284     1
285     1
292     1
293     1
294     1
295     1
296     1
297     1
298     1
299     1
300     1
301     1
302     1
303     1
304     1
305     1
306     1
308     1
309     1
310     1
311     1
342     1
343     1
344     1
345     1
346     1
348     1
349     1
357     1
282     1
278     1


In [69]:
df_cdbg.iloc[537]

Address               230 S JACKSON ST STE 315 SUITE 315
Description                                             
FundedAmount                                     33652.5
IDISActivityID                                      1286
IDISProjectId                                          9
InitialFundingDate                            08/31/2015
PlanYear                                            2015
Project                                      DISPOSITION
Status                                         Completed
StatusDate                                     9/23/2016
Name: 537, dtype: object

In [70]:
sums # all of the repetitions are what we'd expect, looking at index

,Funded Amount,Draw Amount,Balance,FundedAmount
IDIS Activity ID,,,,
1228,37529.44,37529.44,0.0,112588.32
1269,0.00,0.00,0.0,551824.00
1286,33652.54,33652.54,0.0,67305.08
1336,9346.38,9346.38,0.0,56078.28
1394,0.00,0.00,0.0,949432.00
1447,9875.00,9875.00,0.0,29625.00


## Can actually start matching now

In [71]:
df_toadd.head()

,HOUSE_ADDR,CDBG_ADDR,INIT_DATE,STAT_DATE,HOUSE_INDEX,CDBG_INDEX
0,1008 E RESIDENCE AVE,1008 E RESIDENCE AVE,03/08/2008,4/17/2008,0,362
1,1100 SWIFT ST,1100 SWIFT ST,06/03/2008,6/30/2008,1,385
2,1117 W WADDELL AVE,1117 W WADDELL AVE,06/02/2008,6/30/2008,2,382
3,1223 E BROAD AVE,1223 E BROAD AVE,03/08/2008,4/16/2008,3,359
4,1316 E 4TH AVE,1316 E 4TH AVE,06/03/2008,4/9/2009,4,144


In [72]:
df_toadd['HOUSE_INDEX'].tail() # number 563 is missing

558    558
559    559
560    560
561    561
562    562
Name: HOUSE_INDEX, dtype: int64

In [73]:
df_house_cdbg.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
0,26627,NaN,1008 E RESIDENCE AVE,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0,NaN,NaN
1,25739,NaN,1100 SWIFT ST,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.0,NaN,NaN
2,21679,NaN,1117 W WADDELL AVE,2007,5,Energy Efficiency,607,Completed,CDBG,3977.52,3977.52,0.0,NaN,NaN
3,5461,NaN,1223 E BROAD AVE,2007,7,Emergency Repairs,575,Completed,CDBG,4776.14,4776.14,0.0,NaN,NaN
4,25773,NaN,1316 E 4TH AVE,2007,7,Emergency Repairs,613,Completed,CDBG,6934.40,6934.40,0.0,NaN,NaN


In [74]:
init_date = df_toadd['INIT_DATE'].append(pd.Series(["NA"]))
stat_date = df_toadd['STAT_DATE'].append(pd.Series(["NA"]))
init_date.index = range(len(init_date))
stat_date.index = range(len(stat_date))
df_house_cdbg.loc[:,'INIT_DATE'] = init_date
df_house_cdbg.loc[:,'STAT_DATE'] = stat_date

/anaconda3/envs/cds/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [75]:
df_house_cdbg

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,INIT_DATE,STAT_DATE
0,26627,NaN,1008 E RESIDENCE AVE,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.00,NaN,NaN,03/08/2008,4/17/2008
1,25739,NaN,1100 SWIFT ST,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.00,NaN,NaN,06/03/2008,6/30/2008
2,21679,NaN,1117 W WADDELL AVE,2007,5,Energy Efficiency,607,Completed,CDBG,3977.52,3977.52,0.00,NaN,NaN,06/02/2008,6/30/2008
3,5461,NaN,1223 E BROAD AVE,2007,7,Emergency Repairs,575,Completed,CDBG,4776.14,4776.14,0.00,NaN,NaN,03/08/2008,4/16/2008
4,25773,NaN,1316 E 4TH AVE,2007,7,Emergency Repairs,613,Completed,CDBG,6934.40,6934.40,0.00,NaN,NaN,06/03/2008,4/9/2009
5,18199,NaN,1419 W HIGHLAND AVE,2007,5,Energy Efficiency,603,Completed,CDBG,3985.52,3985.52,0.00,NaN,NaN,05/30/2008,6/30/2008
6,21170,NaN,1607 AVALON AVE,2007,7,Emergency Repairs,587,Completed,CDBG,4818.22,4818.22,0.00,NaN,NaN,03/08/2008,4/17/2008
7,11865,NaN,1662 SUNNY LN,2007,5,Energy Efficiency,616,Completed,CDBG,3402.52,3402.52,0.00,NaN,NaN,06/03/2008,6/30/2008
8,25,NaN,1706 S JEFFERSON ST,2007,4,Economic Development,628,Completed,CDBG,2156.32,2156.32,0.00,NaN,NaN,06/06/2008,6/30/2008
9,21214,NaN,1712 W HIGHLAND AVE,2007,5,Energy Efficiency,605,Completed,CDBG,4802.52,4802.52,0.00,NaN,NaN,05/30/2008,6/30/2008


# CDBG is done merging (just remember to check pr03s)

# Starting HOME

In [76]:
df_house_home = df_house[df_house['Program'] == "HOME"]

In [77]:
df_house.shape[0] == df_house_cdbg.shape[0] + df_house_home.shape[0] # quick sanity check

True

In [78]:
df_house_home.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
0,21204,0000Q/00025/009,1707 EDGERLY AVE,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN
1,26763,0000U/00006/002,703 DARTMOUTH LN,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN
2,21680,0000H/00047/005,1115 W WADDELL AVE,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0,NaN,NaN
3,28935,0000X/00003/034,313 FORRESTAL LN,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0,NaN,NaN
4,22320,00240/00003/024,1125 JOEL DR,2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0,NaN,NaN


## read in home data

In [79]:
df_home = pd.read_csv(path + "Raw-Data/homefull.csv")

In [80]:
df_home.head()

,Tenure Type,Activity Type,Grantee,IDIS Activity,Activity Address,Activity Status,Status Date,Total Units,Home Units,Initial Funding Date,Committed Amount,Drawn Amount,PCT
0,Rental,NEW CONSTRUCTION,39508,28,"720 Bonny View Ave , Albany GA, 31705",Completed,6/1/96,4,4,5/8/95,"$171,421.03","$171,421.03",100.00%
1,Rental,NEW CONSTRUCTION,39508,178,"302 Adkins St , Albany GA, 31705",Completed,7/2/97,13,4,7/1/97,"$23,987.70","$23,987.70",100.00%
2,Rental,NEW CONSTRUCTION,39508,285,"700 N Davis St , Albany GA, 31701",Completed,4/1/02,4,4,5/10/01,"$281,081.26","$281,081.26",100.00%
3,Rental,NEW CONSTRUCTION,39508,286,"215 N Broadway St , Albany GA, 31705",Completed,8/1/04,24,24,5/10/01,"$1,804,882.55","$1,804,882.55",100.00%
4,Rental,NEW CONSTRUCTION,39508,367,"208-212 SOUTH MADISON AND 506, 527 HIGHLAND ,...",Completed,11/1/04,6,6,5/31/03,"$534,569.40","$534,569.40",100.00%


In [81]:
s_proj_id = df_house_home['IDIS Activity ID'] #naming is a bit confusing unfortunately
s_home_id = df_home['IDIS Activity']

In [82]:
proj_notin_home = s_proj_id[~s_proj_id.isin(s_home_id)]
home_notin_proj = s_home_id[~s_home_id.isin(s_proj_id)]

In [83]:
home_notin_proj.shape

(119,)

In [84]:
proj_notin_home.shape # yes i am aware i called in house earlier not proj. Also this is more worrisome

(13,)

In [85]:
proj_notin_home # first couple may be related, last 3 might also

13     707
14     708
15     709
16     710
17     711
18     712
19     713
20     714
23     717
88    1116
93    1121
94    1122
95    1123
Name: IDIS Activity ID, dtype: int64

In [86]:
df_house_home[s_proj_id == 707]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
13,4987,000BB/00015/642,431 EBONY LN,2008,19,New Construction,707,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN


In [87]:
df_house_home[s_proj_id == 715]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
21,4894,000BB/00015/615,537 EBONY LN,2008,19,New Construction,715,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
37,10116,0000I/00013/005,1005 MARTIN LUTHER KING JR DR,2009,8,New Construction,715,Completed,HOME,138941.45,138941.45,0.0,NaN,NaN


### So it looks like 707 - 717 are all part of one v large project, but for some reason 715, 716 have extra records

### same case with 1116 - 1123. 1117-1120 have duplicates

### Take some time rn to make sure parsing worked correctly and check those records

### yes, there don't seem to be home records for these values, how about by address?

Okay, so we've found the 706 id record for the northern boundary of ebony l
and also an enormous project of 1115 id record but this time for 125 n broadway st which might match (yup, right next to bonny view)

That solves that... but what about the duplicate records? hmm looks like a coding error on someone at city of albany's part. Typically when you have a large project that is split up, you don't assign separate idis activity ids, you give them each one, esp because that is how they are listed in the pr22. We should definitely ask shelena about this. Looks like she just decided to incrementally assign each project a separate id (more projects) but in doing so, didn't account for duplicates. There might some more duplicates actually...

In [88]:
s_proj_id.value_counts().head(11) # there are 10... we know about 715,716 and 1117-1120. That leaves 1124,1125,1163,1164

1164    2
1124    2
1117    2
1118    2
1119    2
715     2
716     2
1120    2
1125    2
1163    2
1180    1
Name: IDIS Activity ID, dtype: int64

1124,1125,1163,1164 are all listed in the pr22s with blank addresses. They have funded amounts/everything else, but no address. Let's look at what shelena put in the database she gave us:

In [89]:
df_house_home[s_proj_id == 1124]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
96,207,0000C/00005/028,245 BONNY VIEW AVE,2012,20,New Construction,1124,Completed,HOME,158828.16,158828.16,0.0,NaN,NaN
117,30558,0000D/00015/010,603 HALEY ST,2012,10,Tenant Based Rental Assistance,1124,Completed,HOME,13296.00,13296.00,0.0,NaN,NaN


So 1124,1125 are also part of the bonny view project. haley st is not actually in the pr22, but the funded amount is the same

In [90]:
df_house_home[s_proj_id == 1163]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
136,16837,000SS/00001/018,2407 DONCASTER DR,2013,18,Acquisition/Rehabilitation,1163,Completed,HOME,44471.65,44471.65,0.0,NaN,NaN
166,26854,00212/00001/33N,902 LEISURE CT,2013,16,Tenant Based Rental Assistance,1163,Completed,HOME,3296.00,3296.00,0.0,NaN,C


In [91]:
df_house_home[s_proj_id == 1164]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
134,1976,0000I/00008/024,413 CORN AVE,2013,18,Acquisition/Rehabilitation,1164,Completed,HOME,44471.65,44471.65,0.0,NaN,NaN
139,16746,0000S/00013/002,2609 FLYNN CT,2013,16,Tenant Based Rental Assistance,1164,Completed,HOME,7059.00,7059.00,0.0,NaN,NaN


1163,1163 are part of the doncaster project, which is actually coded in the pr22 as 1162. correct values are once again the second record for each idis activity id duplicate

### Fixing id duplicates

#### Ebony project

In [92]:
df_house_home[s_proj_id == 707]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
13,4987,000BB/00015/642,431 EBONY LN,2008,19,New Construction,707,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN


In [93]:
209189 / 19017.18 # from pr22

11.000001051680638

So ids 707 - 717 in the database shelena gave us should all actually be coded under 706

#### broadway / bonny view project

In [94]:
df_house_home[s_proj_id == 1116]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
88,209,0000C/00005/026,250 BONNY VIEW AVE,2012,20,New Construction,1116,Completed,HOME,158828.16,158828.16,0.0,NaN,NaN


In [95]:
1588281.6 / 158828.16

10.0

so ids 1116 - 1125 should all map to 1115

#### Doncaster project

In [96]:
df_house_home[s_proj_id == 1163]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
136,16837,000SS/00001/018,2407 DONCASTER DR,2013,18,Acquisition/Rehabilitation,1163,Completed,HOME,44471.65,44471.65,0.0,NaN,NaN
166,26854,00212/00001/33N,902 LEISURE CT,2013,16,Tenant Based Rental Assistance,1163,Completed,HOME,3296.00,3296.00,0.0,NaN,C


In [97]:
88943.3 / 44471.65

2.0

so ids 1163-1164 should all map to 1162

#### actually fixing--have to be careful with duplicates. Also, we want every large project to be coded the same way, so far, we've only found ones that were coded like this because we ran into dup conflicts. How can we detect large projects?

In [98]:
s_proj_id

0       618
1       634
2       610
3       633
4       600
5       631
6       621
7       599
8       559
9       611
10      601
11      672
12      670
13      707
14      708
15      709
16      710
17      711
18      712
19      713
20      714
21      715
22      716
23      717
24      671
25      669
26      796
27      795
28      783
29      791
30      741
31      784
32      739
33      793
34      794
35      792
36      785
37      715
38      819
39      782
40      716
41      917
42      926
43      923
44      906
45      952
46      922
47      915
48      903
49      904
50      942
51      943
52      929
53      916
54      931
55      913
56      921
57      928
58      934
59      935
60      936
61      937
62     1001
63      933
64     1002
65     1003
66      920
67      914
68      930
69      828
70      884
71      885
72     1031
73     1032
74      996
75      997
76     1006
77     1030
78     1005
79     1108
80     1110
81     1109
82     1089
83  

### One idea is that large projects are coded together in the database. So if we have idis activity numbers going up by one at the same time that indexes are going up, then we should be able to find any multipart projects:

In [99]:
ind = s_proj_id.index
val = s_proj_id.values

In [100]:
multi_proj_indexes = []
prev_ind = ind[0]
prev_val = val[0]
for i in range(1, len(s_proj_id)):
    curr_ind = ind[i]
    curr_val = val[i]
    if (curr_ind == prev_ind + 1) and (curr_val == prev_val + 1):
        if not i-1 in multi_proj_indexes:
            multi_proj_indexes.append(i-1)
        multi_proj_indexes.append(i)
    prev_ind = curr_ind
    prev_val = curr_val

In [101]:
multi_proj_indexes = pd.Series(multi_proj_indexes)
df_house_home.loc[multi_proj_indexes]

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
13,4987,000BB/00015/642,431 EBONY LN,2008,19,New Construction,707,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
14,4958,000BB/00015/619,529 EBONY LN,2008,19,New Construction,708,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
15,4955,000BB/00015/616,535 EBONY LN,2008,19,New Construction,709,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
16,4954,000BB/00015/614,539 EBONY LN,2008,19,New Construction,710,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
17,4938,000BB/00015/611,545 EBONY LN,2008,19,New Construction,711,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
18,4945,000BB/00015/604,559 EBONY LN,2008,19,New Construction,712,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
19,4895,000BB/00015/602,563 EBONY LN,2008,19,New Construction,713,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
20,4957,000BB/00015/618,531 EBONY LN,2008,19,New Construction,714,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
21,4894,000BB/00015/615,537 EBONY LN,2008,19,New Construction,715,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
22,4937,000BB/00015/612,543 EBONY LN,2008,19,New Construction,716,Completed,HOME,19017.19,19017.19,0.0,NaN,NaN


### Not all of these are multi projects, so we're going to do somewhat manually

In [102]:
def identify_multi_projects(df, ids):
    ind = ids.index
    val = ids.values
    multi_proj_indexes = []
    prev_ind = ind[0]
    prev_val = val[0]
    for i in range(1, len(ids)):
        curr_ind = ind[i]
        curr_val = val[i]
        if (curr_ind == prev_ind + 1) and (curr_val == prev_val + 1):
            if not i-1 in multi_proj_indexes:
                multi_proj_indexes.append(i-1)
            multi_proj_indexes.append(i)
        prev_ind = curr_ind
        prev_val = curr_val
    multi_proj_indexes = pd.Series(multi_proj_indexes)
    return df.loc[multi_proj_indexes]

In [103]:
identify_multi_projects(df_house_home, s_proj_id)

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
13,4987,000BB/00015/642,431 EBONY LN,2008,19,New Construction,707,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
14,4958,000BB/00015/619,529 EBONY LN,2008,19,New Construction,708,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
15,4955,000BB/00015/616,535 EBONY LN,2008,19,New Construction,709,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
16,4954,000BB/00015/614,539 EBONY LN,2008,19,New Construction,710,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
17,4938,000BB/00015/611,545 EBONY LN,2008,19,New Construction,711,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
18,4945,000BB/00015/604,559 EBONY LN,2008,19,New Construction,712,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
19,4895,000BB/00015/602,563 EBONY LN,2008,19,New Construction,713,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
20,4957,000BB/00015/618,531 EBONY LN,2008,19,New Construction,714,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
21,4894,000BB/00015/615,537 EBONY LN,2008,19,New Construction,715,Completed,HOME,19017.18,19017.18,0.0,NaN,NaN
22,4937,000BB/00015/612,543 EBONY LN,2008,19,New Construction,716,Completed,HOME,19017.19,19017.19,0.0,NaN,NaN


In [104]:
identify_multi_projects(df_house_cdbg, s_cdbg_id) #check to make sure this is right

/anaconda3/envs/cds/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,INIT_DATE,STAT_DATE
1,25739.0,NaN,1100 SWIFT ST,2007.0,5.0,Energy Efficiency,619.0,Completed,CDBG,4580.04,4580.04,0.00,NaN,NaN,06/03/2008,6/30/2008
2,21679.0,NaN,1117 W WADDELL AVE,2007.0,5.0,Energy Efficiency,607.0,Completed,CDBG,3977.52,3977.52,0.00,NaN,NaN,06/02/2008,6/30/2008
4,25773.0,NaN,1316 E 4TH AVE,2007.0,7.0,Emergency Repairs,613.0,Completed,CDBG,6934.40,6934.40,0.00,NaN,NaN,06/03/2008,4/9/2009
5,18199.0,NaN,1419 W HIGHLAND AVE,2007.0,5.0,Energy Efficiency,603.0,Completed,CDBG,3985.52,3985.52,0.00,NaN,NaN,05/30/2008,6/30/2008
6,21170.0,NaN,1607 AVALON AVE,2007.0,7.0,Emergency Repairs,587.0,Completed,CDBG,4818.22,4818.22,0.00,NaN,NaN,03/08/2008,4/17/2008
7,11865.0,NaN,1662 SUNNY LN,2007.0,5.0,Energy Efficiency,616.0,Completed,CDBG,3402.52,3402.52,0.00,NaN,NaN,06/03/2008,6/30/2008
8,25.0,NaN,1706 S JEFFERSON ST,2007.0,4.0,Economic Development,628.0,Completed,CDBG,2156.32,2156.32,0.00,NaN,NaN,06/06/2008,6/30/2008
9,21214.0,NaN,1712 W HIGHLAND AVE,2007.0,5.0,Energy Efficiency,605.0,Completed,CDBG,4802.52,4802.52,0.00,NaN,NaN,05/30/2008,6/30/2008
14,25964.0,NaN,2002 N SKYLANE DR,2007.0,7.0,Emergency Repairs,556.0,Completed,CDBG,4707.98,4707.98,0.00,NaN,NaN,10/31/2007,4/16/2008
15,18678.0,NaN,207 N INGLESIDE DR,2007.0,7.0,Emergency Repairs,626.0,Completed,CDBG,6932.49,6932.49,0.00,NaN,NaN,06/06/2008,4/1/2009


## Reformatting with no duplicate ids

In [105]:
[6] + [i for i in range(1,10)]

[6, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [106]:
to_fix_home = [[706] + [i for i in range(707,718)], [934,934,935,936,937], [1115] + [i for i in range(1116,1126)],
               [1180] + [i for i in range(1180,1189)]] #check the 1002 1003
to_fix_home_index = [[i for i in range(13, 24)], [58,59,60,61],
                    [i for i in range(88,98)], [i for i in range(150,158)]] # i regret everything. unnecessarily complicated but i'm too far in. please send help

In [107]:
flat_list = [item for sublist in to_fix_home for item in sublist]

In [108]:
to_fix_home

[[706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717],
 [934, 934, 935, 936, 937],
 [1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125],
 [1180, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188]]

In [109]:
def recode_idis_id_multi(df, s_ids, to_fix, to_fix_index):
    flat_list = [item for sublist in to_fix for item in sublist] # stack overflow
    flat_indexes = [item for sublist in to_fix_index for item in sublist]
    s_reg_id = s_ids
    s_multi_id = []
    i = 0
    list_counter = 0
    while i < len(df):
        if s_ids[i] in flat_list and i in flat_indexes:
            part = to_fix[list_counter]
            counter = 0.00
            value = part[0]
            for j in range(1,len(part)):
                s_multi_id.append(value + counter)
                counter += .01
                i = i+1
            list_counter += 1
        else:
            s_multi_id.append(s_ids[i])
            i = i+1
    s_multi_first = [int(t) for t in s_multi_id]
    s_multi_end = [round(t - s,2) for t,s in zip(s_multi_id, s_multi_first)]
    return pd.DataFrame({'OLD_IDIS_ID':s_reg_id,'IDIS_ACTIVITY_ID':s_multi_id, 
                        'IDIS_ACTIVITY_ID_FIRST':s_multi_first, 'IDIS_ACTIVITY_ID_END':s_multi_end})

In [110]:
df_home_toadd = recode_idis_id_multi(df_house_home, s_proj_id, to_fix_home, to_fix_home_index)

In [111]:
df_home_tomatch = pd.concat([df_house_home, df_home_toadd], axis = 1)

In [112]:
df_home_tomatch.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END
0,21204,0000Q/00025/009,1707 EDGERLY AVE,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN,618,618.0,618,0.0
1,26763,0000U/00006/002,703 DARTMOUTH LN,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN,634,634.0,634,0.0
2,21680,0000H/00047/005,1115 W WADDELL AVE,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0,NaN,NaN,610,610.0,610,0.0
3,28935,0000X/00003/034,313 FORRESTAL LN,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0,NaN,NaN,633,633.0,633,0.0
4,22320,00240/00003/024,1125 JOEL DR,2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0,NaN,NaN,600,600.0,600,0.0


In [113]:
s_home_tomatch_ids = df_home_tomatch['IDIS_ACTIVITY_ID_FIRST']

## manual fixing because things are sad

In [114]:
df_home_tomatch.loc[s_home_tomatch_ids == 1163, 'IDIS_ACTIVITY_ID'].iloc[0] = 1162.0
df_home_tomatch.loc[s_home_tomatch_ids == 1163, 'IDIS_ACTIVITY_ID_FIRST'].iloc[0] = 1162
df_home_tomatch.loc[s_home_tomatch_ids == 1163, 'IDIS_ACTIVITY_ID_END'].iloc[0] = 0.0

In [115]:
df_home_tomatch.loc[136,'IDIS_ACTIVITY_ID'] = 1162.0
df_home_tomatch.loc[136,'IDIS_ACTIVITY_ID_FIRST'] = 1162
df_home_tomatch.loc[136,'IDIS_ACTIVITY_ID_END'] = 0.0
df_home_tomatch.loc[134,'IDIS_ACTIVITY_ID'] = 1162.1
df_home_tomatch.loc[134,'IDIS_ACTIVITY_ID_FIRST'] = 1162
df_home_tomatch.loc[134,'IDIS_ACTIVITY_ID_END'] = 0.1

In [116]:
df_home_tomatch['IDIS_ACTIVITY_ID_FIRST'].value_counts().head() # things are finally (?) set

706     11
1115    10
1180     9
934      4
1162     2
Name: IDIS_ACTIVITY_ID_FIRST, dtype: int64

In [117]:
df_home_tomatch.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END
0,21204,0000Q/00025/009,1707 EDGERLY AVE,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN,618,618.0,618,0.0
1,26763,0000U/00006/002,703 DARTMOUTH LN,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN,634,634.0,634,0.0
2,21680,0000H/00047/005,1115 W WADDELL AVE,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0,NaN,NaN,610,610.0,610,0.0
3,28935,0000X/00003/034,313 FORRESTAL LN,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0,NaN,NaN,633,633.0,633,0.0
4,22320,00240/00003/024,1125 JOEL DR,2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0,NaN,NaN,600,600.0,600,0.0


In [118]:
#s_proj_id = df_house_home['IDIS Activity ID'] #naming is a bit confusing unfortunately
#s_home_id = df_home['IDIS Activity']

In [119]:
s_home_id.value_counts().head()

1150    1
1193    1
1184    1
1185    1
1186    1
Name: IDIS Activity, dtype: int64

In [120]:
df_home.head()

,Tenure Type,Activity Type,Grantee,IDIS Activity,Activity Address,Activity Status,Status Date,Total Units,Home Units,Initial Funding Date,Committed Amount,Drawn Amount,PCT
0,Rental,NEW CONSTRUCTION,39508,28,"720 Bonny View Ave , Albany GA, 31705",Completed,6/1/96,4,4,5/8/95,"$171,421.03","$171,421.03",100.00%
1,Rental,NEW CONSTRUCTION,39508,178,"302 Adkins St , Albany GA, 31705",Completed,7/2/97,13,4,7/1/97,"$23,987.70","$23,987.70",100.00%
2,Rental,NEW CONSTRUCTION,39508,285,"700 N Davis St , Albany GA, 31701",Completed,4/1/02,4,4,5/10/01,"$281,081.26","$281,081.26",100.00%
3,Rental,NEW CONSTRUCTION,39508,286,"215 N Broadway St , Albany GA, 31705",Completed,8/1/04,24,24,5/10/01,"$1,804,882.55","$1,804,882.55",100.00%
4,Rental,NEW CONSTRUCTION,39508,367,"208-212 SOUTH MADISON AND 506, 527 HIGHLAND ,...",Completed,11/1/04,6,6,5/31/03,"$534,569.40","$534,569.40",100.00%


In [121]:
df_home[s_home_id==706].index[0]

8

In [122]:
df_home[s_home_id==706]['Status Date'].iloc[0]

'7/19/09'

In [123]:
df_home[df_home['Initial Funding Date'].isna()]

,Tenure Type,Activity Type,Grantee,IDIS Activity,Activity Address,Activity Status,Status Date,Total Units,Home Units,Initial Funding Date,Committed Amount,Drawn Amount,PCT


In [124]:
df_home.iloc[8]

Tenure Type                                                        Rental
Activity Type                                            NEW CONSTRUCTION
Grantee                                                             39508
IDIS Activity                                                         706
Activity Address        400 BLOCK OF THE NORTHERN BOUNDARY OF EBONY L ...
Activity Status                                                 Completed
Status Date                                                       7/19/09
Total Units                                                            55
Home Units                                                             11
Initial Funding Date                                              7/16/09
Committed Amount                                             $209,189.00 
Drawn Amount                                                 $209,189.00 
PCT                                                               100.00%
Name: 8, dtype: object

In [125]:
s_initial_funding_date = []
s_status_funding_date = []
for i in range(len(df_home_tomatch)):
    record = df_home_tomatch.iloc[i]['IDIS_ACTIVITY_ID_FIRST']
    initial = df_home[s_home_id==record]['Initial Funding Date'].iloc[0]
    status = df_home[s_home_id==record]['Status Date'].iloc[0]
    s_initial_funding_date.append(initial)
    s_status_funding_date.append(status)

In [126]:
df_home_append = pd.DataFrame({'INIT_DATE':s_initial_funding_date,
                              'STAT_DATE':s_status_funding_date})

In [127]:
df_home_full = pd.concat([df_home_tomatch, df_home_append], axis=1)

In [128]:
df_cdbg_full = df_house_cdbg

In [129]:
df_home_full.head(2)

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
0,21204,0000Q/00025/009,1707 EDGERLY AVE,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN,618,618.0,618,0.0,6/3/08,9/29/09
1,26763,0000U/00006/002,703 DARTMOUTH LN,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN,634,634.0,634,0.0,7/7/08,12/22/09


In [130]:
df_cdbg_full.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,INIT_DATE,STAT_DATE
0,26627,NaN,1008 E RESIDENCE AVE,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0,NaN,NaN,03/08/2008,4/17/2008
1,25739,NaN,1100 SWIFT ST,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.0,NaN,NaN,06/03/2008,6/30/2008
2,21679,NaN,1117 W WADDELL AVE,2007,5,Energy Efficiency,607,Completed,CDBG,3977.52,3977.52,0.0,NaN,NaN,06/02/2008,6/30/2008
3,5461,NaN,1223 E BROAD AVE,2007,7,Emergency Repairs,575,Completed,CDBG,4776.14,4776.14,0.0,NaN,NaN,03/08/2008,4/16/2008
4,25773,NaN,1316 E 4TH AVE,2007,7,Emergency Repairs,613,Completed,CDBG,6934.40,6934.40,0.0,NaN,NaN,06/03/2008,4/9/2009


In [131]:
old = df_cdbg_full['IDIS Activity ID']
full = df_cdbg_full['IDIS Activity ID'].astype(float)
first = df_cdbg_full['IDIS Activity ID']
end = [0]*564

In [132]:
# error
df_cdbg_full.drop('OLD_IDIS_ID', axis=1, inplace=True)
df_cdbg_full.drop('IDIS_ACTIVITY_ID', axis=1, inplace=True)
df_cdbg_full.drop('IDIS_ACTIVITY_ID_FIRST', axis=1, inplace=True)
df_cdbg_full.drop('IDIS_ACTIVITY_ID_END', axis=1, inplace=True)

KeyError: "['OLD_IDIS_ID'] not found in axis"

In [133]:
df_cdbg_full.insert(14, "OLD_IDIS_ID", old)
df_cdbg_full.insert(15, "IDIS_ACTIVITY_ID", full)
df_cdbg_full.insert(16, "IDIS_ACTIVITY_ID_FIRST", first)
df_cdbg_full.insert(17, "IDIS_ACTIVITY_ID_END", end)

In [134]:
df_cdbg_full.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
0,26627,NaN,1008 E RESIDENCE AVE,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0,NaN,NaN,578,578.0,578,0,03/08/2008,4/17/2008
1,25739,NaN,1100 SWIFT ST,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.0,NaN,NaN,619,619.0,619,0,06/03/2008,6/30/2008
2,21679,NaN,1117 W WADDELL AVE,2007,5,Energy Efficiency,607,Completed,CDBG,3977.52,3977.52,0.0,NaN,NaN,607,607.0,607,0,06/02/2008,6/30/2008
3,5461,NaN,1223 E BROAD AVE,2007,7,Emergency Repairs,575,Completed,CDBG,4776.14,4776.14,0.0,NaN,NaN,575,575.0,575,0,03/08/2008,4/16/2008
4,25773,NaN,1316 E 4TH AVE,2007,7,Emergency Repairs,613,Completed,CDBG,6934.40,6934.40,0.0,NaN,NaN,613,613.0,613,0,06/03/2008,4/9/2009


In [135]:
df_home_full.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
0,21204,0000Q/00025/009,1707 EDGERLY AVE,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN,618,618.0,618,0.0,6/3/08,9/29/09
1,26763,0000U/00006/002,703 DARTMOUTH LN,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN,634,634.0,634,0.0,7/7/08,12/22/09
2,21680,0000H/00047/005,1115 W WADDELL AVE,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0,NaN,NaN,610,610.0,610,0.0,6/2/08,9/29/09
3,28935,0000X/00003/034,313 FORRESTAL LN,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0,NaN,NaN,633,633.0,633,0.0,7/7/08,12/23/09
4,22320,00240/00003/024,1125 JOEL DR,2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0,NaN,NaN,600,600.0,600,0.0,5/19/08,9/29/09


In [136]:
list(df_cdbg_full)

['PrimaryID',
 'Parcel ID',
 'Address',
 'Plan Year',
 'IDIS Project',
 'Project',
 'IDIS Activity ID',
 'Activity Status',
 'Program',
 'Funded Amount',
 'Draw Amount',
 'Balance',
 'Multihome_Type',
 'Multihome',
 'OLD_IDIS_ID',
 'IDIS_ACTIVITY_ID',
 'IDIS_ACTIVITY_ID_FIRST',
 'IDIS_ACTIVITY_ID_END',
 'INIT_DATE',
 'STAT_DATE']

In [137]:
list(df_home_full)

['PrimaryID',
 'Parcel ID',
 'Address',
 'Plan Year',
 'IDIS Project',
 'Project',
 'IDIS Activity ID',
 'Activity Status',
 'Program',
 'Funded Amount',
 'Draw Amount',
 'Balance',
 'Multihome_Type',
 'Multihome',
 'OLD_IDIS_ID',
 'IDIS_ACTIVITY_ID',
 'IDIS_ACTIVITY_ID_FIRST',
 'IDIS_ACTIVITY_ID_END',
 'INIT_DATE',
 'STAT_DATE']

In [138]:
list(df_cdbg_full) == list(df_home_full)

True

In [139]:
df_cdbg_full.shape

(564, 20)

In [140]:
df_home_full.shape

(286, 20)

In [141]:
df_full = pd.concat([df_home_full, df_cdbg_full])

In [142]:
df_full.index = range(len(df_full))

In [143]:
import numpy as np

In [144]:
df_full.iloc[849] = df_full.iloc[849].replace("NA", np.nan)

In [145]:
df_full.tail()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
845,17090,NaN,1601 S SLAPPEY BLVD,2017,5,Section 108 Loan Program,1465,Open,CDBG,26760.00,26760.0,0.00,NaN,NaN,1465,1465.0,1465,0.0,12/06/2017,NaN
846,5224,NaN,230 S JACKSON ST,2017,18,Economic Development,1523,Open,CDBG,242225.78,220057.0,22168.78,NaN,NaN,1523,1523.0,1523,0.0,12/14/2017,NaN
847,20718,NaN,420 PINE AVE,2017,7,Public Service,1477,Open,CDBG,25000.00,25000.0,0.00,NaN,NaN,1477,1477.0,1477,0.0,08/26/2017,NaN
848,5679,NaN,225 W BROAD AVE,2017,7,Public Service,1479,Open,CDBG,25000.00,25000.0,0.00,NaN,NaN,1479,1479.0,1479,0.0,08/26/2017,NaN
849,20726,NaN,514 PINE AVE,2017,22,Commercial Rehabilitation,1547,Open,CDBG,0.00,0.0,0.00,NaN,NaN,1547,1547.0,1547,0.0,NaN,NaN


In [703]:
#df_full.to_csv(path + 'ToDatabase/TotalHouse_v05.csv', index=False)

In [221]:
df_full = pd.read_csv(path + 'ToDatabase/TotalHouse_v05.csv')

In [222]:
df_full[df_full['IDIS_ACTIVITY_ID']==884.0] #delete this record

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
70,26830,000EE/00010/006,701 HOBSON ST,2010,1,Homeowner Rehabilitation,884,Completed,HOME,48245.7,48245.7,0.0,NaN,NaN,884,884.0,884,0.0,12/21/10,11/28/11
501,26830,NaN,701 HOBSON ST,2010,1,Homeowner Rehabilitation,884,Completed,CDBG,0.0,0.0,0.0,NaN,NaN,884,884.0,884,0.0,12/21/2010,11/28/2011


In [223]:
df_full[df_full['IDIS_ACTIVITY_ID']==885.0] # also delete this record

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
71,21174,0000H/00034/001,504 S MAGNOLIA ST,2010,1,Homeowner Rehabilitation,885,Completed,HOME,43588.81,43588.81,0.0,NaN,NaN,885,885.0,885,0.0,12/21/10,4/9/12
495,21174,NaN,504 S MAGNOLIA ST,2010,1,Homeowner Rehabilitation,885,Completed,CDBG,0.00,0.00,0.0,NaN,NaN,885,885.0,885,0.0,12/21/2010,4/9/2012


In [224]:
home = df_full[df_full['Program'] == "HOME"]

In [225]:
cdbg = df_full[df_full['Program'] == "CDBG"]

In [226]:
cdbg.head(5)

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
286,26627,NaN,1008 E RESIDENCE AVE,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0,NaN,NaN,578,578.0,578,0.0,03/08/2008,4/17/2008
287,25739,NaN,1100 SWIFT ST,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.0,NaN,NaN,619,619.0,619,0.0,06/03/2008,6/30/2008
288,21679,NaN,1117 W WADDELL AVE,2007,5,Energy Efficiency,607,Completed,CDBG,3977.52,3977.52,0.0,NaN,NaN,607,607.0,607,0.0,06/02/2008,6/30/2008
289,5461,NaN,1223 E BROAD AVE,2007,7,Emergency Repairs,575,Completed,CDBG,4776.14,4776.14,0.0,NaN,NaN,575,575.0,575,0.0,03/08/2008,4/16/2008
290,25773,NaN,1316 E 4TH AVE,2007,7,Emergency Repairs,613,Completed,CDBG,6934.40,6934.40,0.0,NaN,NaN,613,613.0,613,0.0,06/03/2008,4/9/2009


In [209]:
## DROP COLUMN!!

In [230]:
cdbg.index = range(len(cdbg))

In [232]:
for i in range(len(cdbg)):
    record_stat = cdbg.iloc[i]['STAT_DATE']
    if pd.isna(record_stat):
        print(i)
    else:
        spl = record_stat.split("/")
        year_spl = spl[2]
        if year_spl[0:2] == '20':
            year_spl = year_spl.replace('20',"")
            full_d = spl[0] + "/" + spl[1] + "/" + year_spl
            #print(full_d)
            cdbg.loc[i, 'STAT_DATE'] = full_d

409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
523
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563


In [233]:
cdbg.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
0,26627,NaN,1008 E RESIDENCE AVE,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0,NaN,NaN,578,578.0,578,0.0,03/08/2008,4/17/08
1,25739,NaN,1100 SWIFT ST,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.0,NaN,NaN,619,619.0,619,0.0,06/03/2008,6/30/08
2,21679,NaN,1117 W WADDELL AVE,2007,5,Energy Efficiency,607,Completed,CDBG,3977.52,3977.52,0.0,NaN,NaN,607,607.0,607,0.0,06/02/2008,6/30/08
3,5461,NaN,1223 E BROAD AVE,2007,7,Emergency Repairs,575,Completed,CDBG,4776.14,4776.14,0.0,NaN,NaN,575,575.0,575,0.0,03/08/2008,4/16/08
4,25773,NaN,1316 E 4TH AVE,2007,7,Emergency Repairs,613,Completed,CDBG,6934.40,6934.40,0.0,NaN,NaN,613,613.0,613,0.0,06/03/2008,4/9/09


In [234]:
for i in range(len(cdbg)):
    record = cdbg.iloc[i]['INIT_DATE']
    if pd.isna(record):
        print(i)
    else:
        record_sp = record.split("/")
        month = record_sp[0]
        if month[0] == '0':
            month = month.replace("0","")
            #cdbg.loc[i, 'INIT_DATE'] = month
        year = record_sp[2]
        if year[0:2] == '20':
            year = year.replace("20", "")
        date = month + "/" + record_sp[1] + "/" + year
        cdbg.loc[i, 'INIT_DATE'] = date

563


In [235]:
cdbg.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
0,26627,NaN,1008 E RESIDENCE AVE,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0,NaN,NaN,578,578.0,578,0.0,3/08/08,4/17/08
1,25739,NaN,1100 SWIFT ST,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.0,NaN,NaN,619,619.0,619,0.0,6/03/08,6/30/08
2,21679,NaN,1117 W WADDELL AVE,2007,5,Energy Efficiency,607,Completed,CDBG,3977.52,3977.52,0.0,NaN,NaN,607,607.0,607,0.0,6/02/08,6/30/08
3,5461,NaN,1223 E BROAD AVE,2007,7,Emergency Repairs,575,Completed,CDBG,4776.14,4776.14,0.0,NaN,NaN,575,575.0,575,0.0,3/08/08,4/16/08
4,25773,NaN,1316 E 4TH AVE,2007,7,Emergency Repairs,613,Completed,CDBG,6934.40,6934.40,0.0,NaN,NaN,613,613.0,613,0.0,6/03/08,4/9/09


In [236]:
home.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,OLD_IDIS_ID,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
0,21204,0000Q/00025/009,1707 EDGERLY AVE,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN,618,618.0,618,0.0,6/3/08,9/29/09
1,26763,0000U/00006/002,703 DARTMOUTH LN,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN,634,634.0,634,0.0,7/7/08,12/22/09
2,21680,0000H/00047/005,1115 W WADDELL AVE,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0,NaN,NaN,610,610.0,610,0.0,6/2/08,9/29/09
3,28935,0000X/00003/034,313 FORRESTAL LN,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0,NaN,NaN,633,633.0,633,0.0,7/7/08,12/23/09
4,22320,00240/00003/024,1125 JOEL DR,2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0,NaN,NaN,600,600.0,600,0.0,5/19/08,9/29/09


In [237]:
df_full = pd.concat([home, cdbg])

In [239]:
df_full.drop('OLD_IDIS_ID', axis=1, inplace=True)

In [240]:
df_full.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
0,21204,0000Q/00025/009,1707 EDGERLY AVE,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN,618.0,618,0.0,6/3/08,9/29/09
1,26763,0000U/00006/002,703 DARTMOUTH LN,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN,634.0,634,0.0,7/7/08,12/22/09
2,21680,0000H/00047/005,1115 W WADDELL AVE,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0,NaN,NaN,610.0,610,0.0,6/2/08,9/29/09
3,28935,0000X/00003/034,313 FORRESTAL LN,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0,NaN,NaN,633.0,633,0.0,7/7/08,12/23/09
4,22320,00240/00003/024,1125 JOEL DR,2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0,NaN,NaN,600.0,600,0.0,5/19/08,9/29/09


In [241]:
df_full.to_csv(path + 'ToDatabase/TotalHouse_v06.csv', index=False)

In [242]:
df_full.head()

,PrimaryID,Parcel ID,Address,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome,IDIS_ACTIVITY_ID,IDIS_ACTIVITY_ID_FIRST,IDIS_ACTIVITY_ID_END,INIT_DATE,STAT_DATE
0,21204,0000Q/00025/009,1707 EDGERLY AVE,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN,618.0,618,0.0,6/3/08,9/29/09
1,26763,0000U/00006/002,703 DARTMOUTH LN,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN,634.0,634,0.0,7/7/08,12/22/09
2,21680,0000H/00047/005,1115 W WADDELL AVE,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0,NaN,NaN,610.0,610,0.0,6/2/08,9/29/09
3,28935,0000X/00003/034,313 FORRESTAL LN,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0,NaN,NaN,633.0,633,0.0,7/7/08,12/23/09
4,22320,00240/00003/024,1125 JOEL DR,2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0,NaN,NaN,600.0,600,0.0,5/19/08,9/29/09


In [243]:
#edit manually

In [245]:
cdbg['IDIS Activity ID'].value_counts()

1394    21
1269    21
1336     6
1228     3
1447     3
1286     2
1431     1
1435     1
1434     1
1433     1
1432     1
1426     1
1425     1
1424     1
1423     1
1415     1
1403     1
1436     1
1438     1
1401     1
1439     1
1440     1
1441     1
1442     1
1443     1
1457     1
1458     1
1459     1
1460     1
1461     1
1464     1
1465     1
1470     1
1402     1
1023     1
1399     1
1392     1
1344     1
1345     1
1346     1
1347     1
1349     1
1350     1
1351     1
1352     1
1353     1
1356     1
1357     1
1358     1
1359     1
1360     1
1361     1
1362     1
1363     1
1364     1
1365     1
1366     1
1367     1
1378     1
1379     1
1380     1
1381     1
1382     1
1383     1
1471     1
1472     1
1473     1
1477     1
1548     1
1549     1
1550     1
1551     1
1552     1
1553     1
539      1
540      1
543      1
544      1
546      1
548      1
549      1
550      1
553      1
554      1
555      1
556      1
558      1
560      1
562      1
571      1
572      1